In [1]:
!pip install SpeechRecognition pydub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 14.9 MB/s eta 0:00:00


In [15]:
import speech_recognition as sr
from pydub import AudioSegment
import os
import io
import re
import unicodedata
from IPython.display import Audio

In [16]:
bangla_slang_words = [
    'শালা', 'গাধা', 'পাগল', 'ফালতু', 'হালারপুত', 'চুতমারানি', 'বাজে', 'মাদারচোদ', 'হারামজাদা',
    'কুত্তারবাচ্চা', 'ধুর', 'গরু', 'কুত্তা', 'চুদি', 'চুতিয়া', 'লেংটা', 'গাধামুখো', 'বাচ্চাচোর',
    'হারামি', 'নচ্ছার', 'ভাগাড়েরকুত্তা', 'ধোন', 'লম্পট', 'পাগলা', 'অসভ্য', 'চামার', 'মূর্খ',
    'ফাও', 'ঘোড়া', 'ধেড়ে', 'গণ্ডার', 'বেয়াদব', 'খচ্চর', 'পল্টিবাজ', 'পদলেহনকারী', 'ধুরন্ধর',
    'কিপটে', 'ভানুমতি', 'কামখেয়াল', 'বসন্তেরকুত্তা', 'দুর্দম', 'নালিশবাজ', 'অবাধ্য', 'ছাগল',
    'অপদার্থ', 'খাইস্টা', 'আক্কেলগুড়ুম', 'বিরক্তকর', 'নষ্ট', 'ধড়ফড়ানি', 'হাবাগোবা', 'গোবেচারা',
    'কানাকড়ি', 'অপদ্রব্য', 'নচ্ছার', 'বেল্লিক', 'ঘাড়ত্যাড়া', 'ধড়মড়', 'ভাগার', 'ভেজাল',
    'পিশাচ', 'ভেজাল', 'কপালফাটা', 'ভাদাইম্মা', 'অবাঞ্ছিত', 'পাজি', 'লেংচে', 'কানা', 'খাটাশ',
    'ঠগবাজ', 'বিধ্বংসী', 'বোকাচোদা', 'বদমেজাজি', 'কান্দাছেঁচা', 'বস্তিরছেলে', 'দুশ্চরিত্র',
    'মূর্খ', 'ধুসর', 'ছিন্নমূল', 'বেহায়া', 'পিছল', 'বকচোদা', 'ক্যাঁক', 'অসভ্য', 'পচা',
    'কুটিল', 'কৃপণ', 'দুর্বুদ্ধি', 'বেজন্মা', 'চোর', 'ধাপ্পাবাজ', 'মাদারফাক', 'কামুক', 'চাপাবাজ',
    'ভাগাড়ি', 'বুনো', 'ধামাধার', 'মহিষ', 'ধাঁধাঁ', 'আলফাল', 'ঘাউড়া', 'অলস', 'অবোধ',
    'পক্ক', 'ধমকানি', 'বুদ্ধিহীন', 'গাইয়া', 'ছ্যাঁচড়া', 'অশিক্ষিত', 'মুর্খ', 'ফালতু', 'নখরা',
    'বদরাগী', 'জোচ্চোর', 'ঘুমকাতুরে', 'সুরসুরি', 'কাঠখোট্টা', 'অতৃপ্ত', 'ঠুনকো', 'বাচাল', 'ভন্ড',
    'গাধা', 'অপদার্থ', 'ছ্যাবলা', 'বদমাশ', 'গোলমেলে', 'কুৎসিত', 'খ্যাক', 'খোকারবাচ্চা', 'ফরফরানি',
    'ক্যাঁদাচোদা', 'বদনাম', 'কুরুচিপূর্ণ', 'ধেড়ে', 'বাড়াবাড়ি', 'বাচাল', 'ফাও', 'ঠোঁটকাটা',
    'গামলা', 'ফুর্তিবাজ', 'বাচ্চালোক', 'হাবলু', 'কাচাবাচ্চা', 'বোকাসোকা', 'অরুচিকর', 'খাইস্টা',
    'পাজি', 'অসভ্য', 'ঘ্যানঘ্যান', 'গুজগুজ', 'ঘ্যানঘ্যানে', 'ফকিন্নি', 'ভোতকা', 'পুটকি', 'লেঙ্গা',
    'ভাঁড়', 'বাঁদর', 'খাউড়', 'গুঁফো', 'ধূর্ত', 'ফাঁকিবাজ', 'অজগর', 'চোটপাট', 'মাতাল', 'কাবুলিওয়ালা',
    'কিম্ভূত', 'কদাকার', 'ছেলেখেলা', 'বখাটে', 'মাথামোটা', 'অকালপক্ক', 'পুটুসপাটাস', 'বাচাল',
    'খিটখিটে', 'হাঁপাচ্ছি', 'বাচ্চাবাচ্চি', 'গোবর', 'কিম্ভূতকিমাকার', 'ফালতু', 'বেলেল্লাপনা',
    'পাঁচফোড়ন', 'বেহিসেবী', 'ধড়ফড়ানি', 'প্যাঁচারবাচ্চা', 'ফাঁকিবাজ', 'লুচ্চা', 'ফাকিবাজ',
    'খাউড়া', 'বেওকুফ', 'মাতাল', 'নেত্যাচ্ছি', 'কঞ্জুস', 'ঘ্যানঘ্যানানি', 'গজদন্ত', 'নেকামি',
    'কুলাঙ্গার', 'ঘ্যানঘ্যান', 'পুটুসপাটাস', 'ফাজিল', 'ধুরন্ধর', 'গরিবি', 'কিপ্টা', 'কিপটে',
    'আওল', 'ঘাপটি', 'গাদ্দার', 'বাকবাকুম', 'চোপস', 'অগোছালো', 'প্যাঁচাল', 'ফচকে', 'চামচা',
    'ভ্যাবলা', 'চটকবাজ', 'চ্যালা', 'ফাঁকিবাজ', 'কপচানো', 'ভাঁড়', 'গাধামুখো', 'পেটুক', 'ঘ্যানঘ্যান',
    'বাজে', 'বাচাল', 'কামুক', 'লম্পট', 'বেজন্মা', 'ফাকিবাজ', 'মাকাল', 'কৃপণ', 'কানা', 'কুপমণ্ডুক',
    'ঘ্যানঘ্যানানি', 'কুৎসা', 'অভাবী', 'আলসেমি', 'মাথামোটা', 'কানা', 'অসভ্য', 'পদলেহন', 'ফুটানি',
    'কুটিল', 'কৃপণ', 'ধেড়ে', 'ছ্যাবলা', 'কুৎসিত', 'বাচাল', 'ঘাউড়া', 'কুটনামি', 'ফাকিবাজ', 'ঠকবাজ',
    'বদমাশ', 'মাতাল', 'অসভ্য', 'গাধা', 'বদরাগী', 'ফাঁকিবাজ', 'ভেজাল', 'নিমকহারাম', 'ধামাধার',
    'বখাটে', 'নেড়ি', 'ফার্মেরমুরগি', 'প্যাঁচাল', 'গালগল্প', 'বকবক', 'গালমন্দ', 'খুঁতখুঁতে', 'মূর্খ',
    'খিচুড়ি', 'বাজে', 'গাইয়া', 'নোংরা', 'বাবুঘোষ', 'কুচুটে', 'খামখেয়ালী', 'গরুরগাড়ি', 'পঁচা',
    'মাথাখারাপ', 'প্যাঁচাল', 'খিচুড়ি', 'ফালতু', 'লম্পট', 'খানকির ছেলে', 'লুচ্চা', 'খচ্চর', 'গাধা',
    'বদমাশ', 'প্যাঁচাল', 'গোমূর্খ', 'ঘ্যানঘ্যান', 'গাধা', 'বকবক', 'নোংরা', 'কুৎসা', 'গালভরা', 'ফালতু',
    'খুচরো', 'ফুঁকানি', 'ফাজিল', 'নিমকহারাম', 'ফাজিল', 'গাধা', 'কুৎসিত', 'ছ্যাবলা', 'বাজে', 'মাথাখারাপ',
    'ঘ্যাচঘ্যাচ', 'মূর্খ', 'ফালতু', 'প্যাঁচাল', 'অবাধ্য', 'মূর্খ', 'গাইয়া', 'বাচাল', 'গরুরগাড়ি','কুত্তাচোদা','ডাইনাসর চোদা']


In [35]:
def play_audio(file_path):
    display.display(Audio(file_path))

In [36]:
def convert_audio_to_wav(file_path):
    audio = AudioSegment.from_file(file_path)
    wav_file_path = file_path.replace(".mp3", ".wav")
    audio.export(wav_file_path, format="wav")
    return wav_file_path

In [37]:
def transcribe_audio(file_path):
    recognizer = sr.Recognizer()
    with sr.AudioFile(file_path) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data, language="bn-BD")
            return text
        except sr.UnknownValueError:
            return ""
        except sr.RequestError as e:
            return ""

In [38]:
def normalize_unicode(text):
    return unicodedata.normalize('NFC', text)

In [39]:
def detect_slang(text):
    detected_slangs = []

    normalized_text = normalize_unicode(text)
    normalized_slang_words = [normalize_unicode(slang) for slang in bangla_slang_words]

    for slang in normalized_slang_words:
        if slang in normalized_text:
            detected_slangs.append(slang)

    return detected_slangs

In [40]:
def bus_control(file_path):
    if file_path.endswith(".mp3"):
        file_path = convert_audio_to_wav(file_path)
        play_audio(file_path)
    transcription = transcribe_audio(file_path)

    if transcription:
        print(f"Transcribed Text: {transcription}")
    else:
        print("No transcribable audio found.")

    slang_detected = detect_slang(transcription)

    if slang_detected:
        print("Detected Slang Words:", slang_detected)
        print("Bus is running!")
    else:
        print("No slang detected. Bus remains still.")

In [49]:
from google.colab import files
uploaded = files.upload()

Saving m3.mp3 to m3.mp3


In [50]:
for filename in uploaded.keys():
    bus_control(filename)

Transcribed Text: মূর্খ শালা টান দে
Detected Slang Words: ['শালা', 'মূর্খ', 'মূর্খ', 'মূর্খ', 'মূর্খ', 'মূর্খ']
Bus is running!
